In [1]:
#Relevant imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math
import random
import io
from PIL import Image
import requests
from bs4 import BeautifulSoup
import json
import time

#Required imports to create button widgets
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from jupyter_ui_poll import ui_events

In [2]:
#Finding coordinates for a point in an ellipse plot
def random_point_in_ellipse(center_x, center_y, width, height):
    h = center_x
    k = center_y
    a = width / 2
    b = height / 2

    theta = random.uniform(0, 2 * math.pi)
    while True:
        r = random.uniform(0, 1)
        x = a * r * math.cos(theta)
        y = b * r * math.sin(theta)
        if (x/a)**2 + (y/b)**2 <= 1:
            break

    return h + x, k + y

#Function to separate dot coordinates
#The dots were overlapping
#Needed to overcome the issue
def is_too_close(point, others, min_distance):
    x, y = point
    for ox, oy in others:
        if math.sqrt((x - ox)**2 + (y - oy)**2) < min_distance:
            return True
    return False

#Function to generate the actual dots
#On the previously defined coordinates
def generate_dot_image(blue_dots, yellow_dots, filename):
    fig, ax = plt.subplots()
    markersize = 15  # Set markersize
    min_distance = markersize * 0.005  # Minimum distance between dots

    ellipse1 = patches.Ellipse((0.25, 0.5), 0.48, 0.98, angle=0, edgecolor='black', facecolor='none')
    ellipse2 = patches.Ellipse((0.75, 0.5), 0.48, 0.98, angle=0, edgecolor='black', facecolor='none')

    blue_points = []
    yellow_points = []

    # For blue dots
    for _ in range(blue_dots):
        attempts = 0
        while attempts < 100:
            x, y = random_point_in_ellipse(0.25, 0.5, 0.4, 0.9)
            if not is_too_close((x, y), blue_points, min_distance):
                blue_points.append((x, y))
                ax.plot(x, y, 'o', color='blue', markersize=markersize)
                break
            attempts += 1

    # For yellow dots
    for _ in range(yellow_dots):
        attempts = 0
        while attempts < 100:
            x, y = random_point_in_ellipse(0.75, 0.5, 0.40, 0.9)
            if not is_too_close((x, y), yellow_points, min_distance):
                yellow_points.append((x, y))
                ax.plot(x, y, 'o', color='#F6C242', markersize=markersize)
                break
            attempts += 1

    # Add the ellipses to the axes
    ax.add_patch(ellipse1)
    ax.add_patch(ellipse2)

    #Used AI tools to 
    # Set the aspect of the plot to be equal
    ax.set_aspect('auto')
    
    # Set limits and turn off axes
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')

    plt.savefig(filename+'.png')
    plt.close()

In [3]:
#Generating the set of 64 images
#Using the ratios provided, create 8 unique images
#Then flip the images, switching the colours
#This produces 16 images
#Then the loop is repeated 4 times
#To generate 64 images with different dot spatial arrangement
def gen_images():
    counter = 1
    
    for j in range(4):
        c = 3
        for i in range(3):
            generate_dot_image(c*4,c*3, str(counter))
            counter+=1
            c+=1

        c = 2
        for i in range(2):
            generate_dot_image(c*7,c*6, str(counter))
            counter+=1
            c+=1

        c = 1
        for i in range(2):
            generate_dot_image(c*10,c*9, str(counter))
            counter+=1
            c+=1

        generate_dot_image(18,16, str(counter))
        counter+=1

        c = 3
        for i in range(3):
            generate_dot_image(c*3,c*4, str(counter))
            counter+=1
            c+=1

        c = 2
        for i in range(2):
            generate_dot_image(c*6,c*7, str(counter))
            counter+=1
            c+=1

        c = 1
        for i in range(2):
            generate_dot_image(c*9,c*10, str(counter))
            counter+=1
            c+=1

        generate_dot_image(16,18, str(counter))
        counter+=1

#Function to generate the corresponding answer key based on the number of dots
def gen_answer():
  answers = []
  for k in range(4):
    for i in range(8):
      answers.append("Left")

    for j in range(8):
      answers.append("Right")

  return answers

#Returns an array of all 64 images
#This is subsequently displayed to the user in the test
def get_image():
    counter = 1
    gen_images()
    images = []
    for i in range(64):
        image_path = str(counter) + ".png"
        counter+=1
        img = Image.open(image_path).convert('RGB')  # Convert to RGB
        if img is not None:
            images.append(img)
    return images

answ = gen_answer()

In [4]:
#Function to code for answer inputs being submitted directly on a Google Form
def send_to_google_form(data_dict, form_url):
    ''' Helper function to upload information to a corresponding google form 
        You are not expected to follow the code within this function!
    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
 
    #Code for if form data does not match input data
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

In [5]:
event_info = {
    'type': '',
    'description': '',
    'time': -1
}

#Create button widgets for a specific time interval
def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):
    start_wait = time.time()

    # set event info to be empty
    # as this is dict we can change entries
    # directly without using
    # the global keyword
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1

    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False

            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False

            # add pause before looping
            # to check events again
            time.sleep(interval)

    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info

# this function lets buttons
# register events when clicked
def register_event(btn):
    # display button description in output area
    event_info['type'] = "click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return


In [6]:
# Function to show the images one by one with specified display time and response time
def show_images(image_files, display_time=0.75, response_time=3):

    answer = gen_answer()
    #Creating an empty responses list 
    #To append specific user responses into
    responses = []

    #Collection of user consent
    data_consent_info = """DATA CONSENT INFORMATION:

    Please read:

    we wish to record your response data

    to an anonymised public data repository.

    Your data will be used for educational teaching purposes

    practising data analysis and visualisation.

    Please type yes in the box below if you consent to the upload."""

    print(data_consent_info)
    result = input("> ")
    #Creating an if statement providing further contact information if the user consents to participating
    if (result == "yes" or result == "YES"):

        print("Thanks for your participation.")

        print("Please contact philip.lewis@ucl.ac.uk")

        print("If you have any questions or concerns")

        print("regarding the stored results.")

    #If the user does not consent, the test will not continue after printing this statement
    else:

        raise(Exception("User did not consent to continue test."))

    #To enhance the user interface, these lines of code make the text disappear after the user has given the appropriate response
    time.sleep(3)
    clear_output(wait=True)

    #Explaining the requirements for creation of a User ID
    print("""

    Enter your anonymised ID

    To generate an anonymous 4-letter unique user identifier please enter:

    - two letters based on the initials (first and last name) of a childhood friend

    - two letters based on the initials (first and last name) of a favourite actor

    Example: if your friend is named David Johnson and favourite actor is Morgan Freeman

    then your unique identifer would be DJMF

    """)
    user_id = input("> ")

    #Instructions only disappear once the user has understood the instructions and created a relevant User ID
    time.sleep(1.5)
    clear_output(wait=True)

    #Displaying the User ID to begin the test
    print("User entered id:", user_id)

    #Displaying the instructions of the test
    print(f"Welcome to the Approximate Number System (ANS) test {user_id}.\n This test evaluates your non-verbal numerical estimation skills.\n The ANS is a portion of your congition that is active throughout your life. \n During this test, you will be shown 64 images, \n with blue dots on the left and yellow dots on the right. \n Each image has a different number of dots on both sides. \n Your task is to simply guess which side has more dots, by clicking the Left or Right button\n At the end, your score will be calculated.")
    time.sleep(3)
    clear_output(wait=True)

    #Giving the user a brief period before starting to enhance user interface
    print("Ready? Let's begin")
    time.sleep(3)
    clear_output(wait=True)

    #Displaying all the images in the array one-by-one
    for i in range(64):
        img = image_files[i]
        display(img)
        
        time.sleep(display_time)
        clear_output(wait=True)

        btn1 = widgets.Button(description="Left")
        btn2 = widgets.Button(description="Right")

        #Setting up the buttons by calling on the function
        btn1.on_click(register_event)
        btn2.on_click(register_event)

        #Displaying the question after each image and allowing the user to answer within the time frame
        myhtml1 = HTML("<h1>Which side has more dots</h1>")
        display(myhtml1)
        myhtml2 = HTML("<h2>You have 3 seconds to answer</h2>")
        display(myhtml2)

        panel = widgets.HBox([btn1, btn2])
        display(panel)

        result = wait_for_event(timeout=response_time)
        
        clear_output(wait=True)

        #Giving a correct or incorrect score only if a button was clicked, otherwise storing it as an unanswered question.
        if result['description']!="":
            responses.append(result['description'])
            if result['description']== answer[i]:
                myhtml3 = HTML("<h2>Correct</h2>")
                display(myhtml3)
                time.sleep(1.5)
                clear_output(wait=True)

            else:
                myhtml3 = HTML("<h2>Incorrect</h2>")
                display(myhtml3)
                time.sleep(1.5)
                clear_output(wait=True)
        else:
            responses.append("User did not click in time")
        clear_output(wait=True)

    #Finally submitting all responses to the Google Form
    form_url = "https://docs.google.com/forms/d/e/1FAIpQLSfpWUE9GWZ1I22dOzkwdQqt9P9xJM2YwQgHOT3I7hqyoqnU3A/viewform?usp=sf_link"
    mydata = calculate_score(user_id, answer, responses)
    send_to_google_form(mydata, form_url)
    
    
    #Letting the user know upon completion of the test
    print(f"Your total score is: {mydata['Score']}.\n Thank you for participating in the ANS test \n Your responses have been recorded for our investigation")
    time.sleep(5)
    clear_output(wait=True)
    
    return


In [7]:
#Function to calculate the total score
#Giving +1 for every correct answer, -1 for every incorrect answer and 0 for unanswered questions
def calculate_score(user_id, all_answers, responses):
    right = 0
    wrong = 0
    unanswered = 0
    score = []
    for i in range(64):
        if(all_answers[i] == responses[i]):
            right += 1
        elif  (responses[i] == "User did not click in time"):
            unanswered +=1
        else:
            wrong+=1
    score = right-wrong

    #Storing the respective scores in the responses section for a specific question on the Google Form
    mydata = {
    'User ID': user_id,
    'Correct': right,
    'Incorrect': wrong,
    'Unanswered': unanswered,
    'Score': score
}
    return mydata

In [8]:
#Allowing the user to run the test
show_images(get_image(), display_time=0.75)

Your total score is: 27.
 Thank you for participating in the ANS test 
 Your responses have been recorded for our investigation
